In [1]:
import json 
data = [json.loads(line) for line in open('subset_well_parsed.json', 'r')]

In [2]:
spanish_compatible_data = [d for d in data if d["fulltext_html_es"] is not None]
len(spanish_compatible_data)

996

In [3]:
spanish_compatible_data[0].keys()

dict_keys(['_id', 'ab_pt', 'fulltext_html_es', 'fulltext_pdf_en', 'ab', 'ti_en', 'journal_title', 'wok_subject_categories', 'keyword_es', 'fulltext_html_pt', 'da', 'subject_area', 'au', 'sponsor', 'available_languages', 'keyword_en', 'ti_es', 'arquivo', 'fulltext_html_en', 'fulltext_pdf_es', 'ti_pt', 'doi', 'ab_es', 'la', 'fulltext_pdf_pt', 'ur', 'wok_citation_index', 'keyword_pt', 'version', 'ti', 'ab_en', 'type', 'use_license', 'ta', 'body_content_en', 'body_content_es', 'body_content_pt'])

In [4]:
scientific_text_data = list()
for i,d in enumerate(spanish_compatible_data):
    spanish_text = str(d["body_content_es"])
    if "body_content_en" not in d:
        continue
    english_text = str(d["body_content_en"])
    article_object = {"en":english_text,"es":spanish_text}
    scientific_text_data.append(article_object)
scientific_text_data[0]

{'en': '{\'Description of evidence collection method\': [\'We conducted searches in multiple databases (Medline 1965-2012; Cochrane Library,               Lilacs) and cross-references with the collected material to identify studies with               better methodological design, followed by a critical evaluation of their contents and               classification according to the strength of evidence.\', \'We conducted searches between August and December 2012. The following strategies were               used for searches in PubMed:\', \'"regional anaesthesia" OR "anesthesia, conduction" OR "anesthesia" AND                     "conduction" OR "conduction anesthesia" OR "regional" AND "anesthesia" OR                     "regional anesthesia" AND "antithrombotic";\', \'"regional anaesthesia" OR "anesthesia, conduction" [MeSH Terms] AND "infection"                     [MeSH Terms] AND "thromboembolism" [MeSH Terms] OR "thromboembolism" [All                     Fields];\', \'"thromboemboli

In [5]:
import json
with open("scientific_text_data.json","w") as scd:
    json.dump(scientific_text_data,scd)

In [6]:
from datasets import load_dataset_builder,load_dataset

C:\Users\juan9\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [7]:
ds_builder = load_dataset_builder("xquad","xquad.en")

In [8]:
ds_builder.info.features

{'id': Value(dtype='string', id=None),
 'context': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}

In [9]:
english_xquad = load_dataset("xquad","xquad.en",split="validation")

In [10]:
english_xquad[0]

{'id': '56beb4343aeaaa14008c925b',
 'context': "The Panthers defense gave up just 308 points, ranking sixth in the league, while also leading the NFL in interceptions with 24 and boasting four Pro Bowl selections. Pro Bowl defensive tackle Kawann Short led the team in sacks with 11, while also forcing three fumbles and recovering two. Fellow lineman Mario Addison added 6½ sacks. The Panthers line also featured veteran defensive end Jared Allen, a 5-time pro bowler who was the NFL's active career sack leader with 136, along with defensive end Kony Ealy, who had 5 sacks in just 9 starts. Behind them, two of the Panthers three starting linebackers were also selected to play in the Pro Bowl: Thomas Davis and Luke Kuechly. Davis compiled 5½ sacks, four forced fumbles, and four interceptions, while Kuechly led the team in tackles (118) forced two fumbles, and intercepted four passes of his own. Carolina's secondary featured Pro Bowl safety Kurt Coleman, who led the team with a career high se

In [11]:
unique_english_texts = list()
for text_object in english_xquad:
    if text_object["context"] not in unique_english_texts:
        unique_english_texts.append(text_object["context"])

In [12]:
len(unique_english_texts)

240

In [13]:
spanish_xquad = load_dataset("xquad","xquad.es",split="validation")

In [14]:
unique_spanish_texts = list()
for text_object in spanish_xquad:
    if text_object["context"] not in unique_spanish_texts:
        unique_spanish_texts.append(text_object["context"])

In [15]:
len(unique_spanish_texts)

240

In [16]:
list_unique_spanish_texts = unique_spanish_texts
list_unique_english_texts = unique_english_texts


In [17]:
unique_text_objects = list()
for i in range(len(list_unique_english_texts)):
    text_object = {
        "en": list_unique_english_texts[i],
        "es": list_unique_spanish_texts[i]
    }
    unique_text_objects.append(text_object)

unique_text_objects[0]

{'en': "The Panthers defense gave up just 308 points, ranking sixth in the league, while also leading the NFL in interceptions with 24 and boasting four Pro Bowl selections. Pro Bowl defensive tackle Kawann Short led the team in sacks with 11, while also forcing three fumbles and recovering two. Fellow lineman Mario Addison added 6½ sacks. The Panthers line also featured veteran defensive end Jared Allen, a 5-time pro bowler who was the NFL's active career sack leader with 136, along with defensive end Kony Ealy, who had 5 sacks in just 9 starts. Behind them, two of the Panthers three starting linebackers were also selected to play in the Pro Bowl: Thomas Davis and Luke Kuechly. Davis compiled 5½ sacks, four forced fumbles, and four interceptions, while Kuechly led the team in tackles (118) forced two fumbles, and intercepted four passes of his own. Carolina's secondary featured Pro Bowl safety Kurt Coleman, who led the team with a career high seven interceptions, while also racking up

In [18]:
with open("QA_parallel_dataset.json","w") as qa:
    json.dump(unique_text_objects,qa)

In [1]:
import json
scientific_text_data = list()
with open("scientific_text_data.json","r") as st:
    scientific_text_data = json.load(st)
len(scientific_text_data)

995

In [2]:
from nltk.corpus import stopwords
import nltk
import re
from sklearn.feature_extraction.text import CountVectorizer
import spacy
from unidecode import unidecode


class TextPreparation(object):
    def __init__(self,text_series,cv_params={"min_df":0.001,"max_df":0.7},language="english"):
        self.text_series = text_series
        self.cv_params = cv_params
        self.language=language
        language_model_name_dict = {"english":"en_core_web_sm","spanish":"es_core_news_sm"}
        self.model = spacy.load(language_model_name_dict[self.language], disable = ['parser','ner'])

    def _get_stopwords(self):
        nltk.download("stopwords")
        english_stopwords = stopwords.words(self.language)
        self.stopwords = english_stopwords

    
    def _clean_text(self,text):
        self._get_stopwords()
        lower_text = text.str.lower()
        no_accents = lower_text.apply(unidecode)
        alpha_text =  no_accents.str.replace(r"(@\[a-z]+)|([^a-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "")
        nostops = alpha_text.apply(lambda x: " ".join([word for word in x.split() if word not in self.stopwords]))
        return nostops
    
    def _lemmatize_words(self,text):
        def _lemmatize_doc(doc):
            doc_obj = self.model(doc)
            lemmatized = " ".join([token.lemma_ for token in doc_obj])
            return lemmatized
        return text.apply(_lemmatize_doc)
    
    def _filter_words(self,text):
        cv = CountVectorizer(min_df=self.cv_params["min_df"],max_df=self.cv_params["max_df"])
        cv.fit(text)
        filtered_text = text.apply(lambda x: " ".join([word for word in x.split() if word in cv.vocabulary_]))
        return filtered_text
    
    
    def prepare_text(self,pipeline=["clean","lemmatize","filter"]):
        functions = {"clean":self._clean_text,"filter":self._filter_words,"lemmatize":self._lemmatize_words}
        text = self.text_series
        for step in pipeline:
            text = functions[step](text)
        
        return text



<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
import gc;gc.collect()
import pandas as pd

C:\Users\juan9\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [4]:
unique_text_df = pd.DataFrame(unique_text_objects)
text_prep_en = TextPreparation(unique_text_df.en)

NameError: name 'unique_text_objects' is not defined

In [ ]:
unique_text_df

,en,es
0,"The Panthers defense gave up just 308 points, ...","﻿Los Panthers, que además de liderar las inter..."
1,The Broncos defeated the Pittsburgh Steelers i...,Los Broncos vencieron a los Pittsburgh Steeler...
2,Peyton Manning became the first quarterback ev...,Peyton Manning se convirtió en el primer maris...
3,Six-time Grammy winner and Academy Award nomin...,"Lady Gaga, la seis veces ganadora del Grammy y..."
4,"With 4:51 left in regulation, Carolina got the...",Con 4 min 51 s de tiempo restante en el marcad...
...,...,...
235,Aristotle provided a philosophical discussion ...,Aristóteles ofreció una discusión filosófica d...
236,The development of fundamental theories for fo...,El desarrollo de teorías fundamentales para la...
237,"Since then, and so far, general relativity has...","Desde entonces, y hasta ahora, la relatividad ..."
238,Through combining the definition of electric c...,Mediante la combinación de la definición de co...


In [ ]:
print(unique_text_df.en.values[0])

The Panthers defense gave up just 308 points, ranking sixth in the league, while also leading the NFL in interceptions with 24 and boasting four Pro Bowl selections. Pro Bowl defensive tackle Kawann Short led the team in sacks with 11, while also forcing three fumbles and recovering two. Fellow lineman Mario Addison added 6½ sacks. The Panthers line also featured veteran defensive end Jared Allen, a 5-time pro bowler who was the NFL's active career sack leader with 136, along with defensive end Kony Ealy, who had 5 sacks in just 9 starts. Behind them, two of the Panthers three starting linebackers were also selected to play in the Pro Bowl: Thomas Davis and Luke Kuechly. Davis compiled 5½ sacks, four forced fumbles, and four interceptions, while Kuechly led the team in tackles (118) forced two fumbles, and intercepted four passes of his own. Carolina's secondary featured Pro Bowl safety Kurt Coleman, who led the team with a career high seven interceptions, while also racking up 88 tack

In [ ]:
prepared_text_en_qa = text_prep_en.prepare_text()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\juan9\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-19-46b20e55a32d>:27: FutureWarning: The default value of regex will change from True to False in a future version.
  alpha_text =  no_accents.str.replace(r"(@\[a-z]+)|([^a-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "")


In [ ]:
prepared_text_en_qa.head()

0    panther defense give point rank sixth league a...
1    broncos defeat pittsburgh steeler divisional r...
2    peyton manning become first quarterback ever l...
3    sixtime grammy winner academy award nominee la...
4    leave regulation carolina get ball yard line c...
Name: en, dtype: object

In [ ]:
text_prep_es = TextPreparation(unique_text_df.es,language="spanish")
prepared_text_es_qa = text_prep_es.prepare_text()
prepared_text_es_qa

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\juan9\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-19-46b20e55a32d>:27: FutureWarning: The default value of regex will change from True to False in a future version.
  alpha_text =  no_accents.str.replace(r"(@\[a-z]+)|([^a-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "")


0      panther adema liderar intercepción nfl contar ...
1      bronco vencer pittsburgh steelers ronda divisi...
2      peyton manning convirtio primero mariscal camp...
3      lady gagar seis vez ganador grammy nominado os...
4      min tiempo restante marcador carolín atrapo ba...
                             ...                        
235    aristotel ofrecio discusion filosofico concept...
236    desarrollo teoria fundamental fuerza procedio ...
237    entonces ahora relatividad general reconocido ...
238    mediante combinacion definicion corriente elec...
239    area seccion transversal relevante volumen cal...
Name: es, Length: 240, dtype: object

In [6]:
scientific_text_df = pd.DataFrame(scientific_text_data)
scientific_text_df.shape

(995, 2)

In [7]:
scientific_text_df.head()

,en,es
0,{'Description of evidence collection method': ...,{'Descripción del método de recolección de evi...
1,{'Introduction': ['Anxiety is a natural reacti...,{'Introducción': ['La ansiedad es una reacción...
2,"{'Introduction': [""Postoperative complication ...",{'Introducción': ['Se considera una complicaci...
3,{'Introduction': ['Mivacurium chloride is a sh...,{'Introducción': ['El cloruro de mivacurio es ...
4,{'Introduction': ['Pulmonary hypertension (PH)...,{'Introducción': ['La hipertensión pulmonar (H...


In [8]:
text_prep_en = TextPreparation(scientific_text_df.en)
prepared_text_en_science = text_prep_en.prepare_text()
prepared_text_en_science.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\juan9\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-2-46b20e55a32d>:27: FutureWarning: The default value of regex will change from True to False in a future version.
  alpha_text =  no_accents.str.replace(r"(@\[a-z]+)|([^a-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "")


0    description evidence collection conduct search...
1    anxiety natural reaction arise response enter ...
2    postoperative complication occurrence unexpect...
3    mivacurium chloride short act nondepolarize ne...
4    pulmonary hypertension ph chronic disease defi...
Name: en, dtype: object

In [9]:
text_prep_es = TextPreparation(scientific_text_df.es,language="spanish")
prepared_text_es_science = text_prep_es.prepare_text()
prepared_text_es_science.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\juan9\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-2-46b20e55a32d>:27: FutureWarning: The default value of regex will change from True to False in a future version.
  alpha_text =  no_accents.str.replace(r"(@\[a-z]+)|([^a-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "")


0    descripcion recoleccion evidencia busqueda mul...
1    ansiedad reaccion natural origín respuesta ent...
2    complicacion postoperatoria alteracion inesper...
3    cloruro mivacurio agente neuromuscular despola...
4    hipertension pulmonar hp enfermedad cronico de...
Name: es, dtype: object

In [10]:
prepared_text_es_science.values[0]

'descripcion recoleccion evidencia busqueda multipl base medline cochrane library lilacs referencia cruzado material investigado identificacion articulo mejor diseno metodologico seguido posterior evaluacion criticar contenido clasificacion tono fuerza evidenciar busqueda agosto diciembre busqueda pubmed usar siguiente estrategia regional anaesthesia or anesthesia conduction or anesthesia and conduction or conduction anesthesio or regional and anesthesio or regional anesthesia and antithrombotic regional anaesthesia or anesthesio conduction mesh terms and infection mesh terms and thromboembolism mesh terms or thromboembolism all fields thromboembolism mesh terms or thromboembolism all fields and regional anaesthesia all fields or anesthesio conduction mesh terms or anesthesio all fields and conduction all fields or conduction anesthesia all fields or regional all fields and anesthesia all fields or regional anesthesia all fields campo anestesia regional seleccionar abordar conducta rea

In [11]:
from sentence_transformers import SentenceTransformer

In [12]:
from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize
import numpy as np

class TextEmbeddingGenerator(object):

    def __init__(self,texts,model='all-MiniLM-L6-v2'):
        self.model = model
        self.texts = texts

    def _load_transformer(self):
        self.transformer = SentenceTransformer(self.model)

    def unload_transformer(self):
        del self.transformer


    def _generate_text_embeddings(self):
        text_embeddings = list()
        for text in self.texts.values:
            tokenized_sentences = sent_tokenize(text)
            sentence_embeddings = self.transformer.encode(tokenized_sentences)
            averaged_sentences = sentence_embeddings.mean(axis=0)
            text_embeddings.append(averaged_sentences)
        
        return np.array(text_embeddings)
    

    def calculate_embeddings(self):
        self._load_transformer()
        embeddings = self._generate_text_embeddings()
        return embeddings
    

        
        

In [13]:
embedding_gen = TextEmbeddingGenerator(prepared_text_en_science,"paraphrase-multilingual-MiniLM-L12-v2")

In [14]:
embeddings = embedding_gen.calculate_embeddings()

In [15]:
embedding_gen.unload_transformer()

In [16]:
del embedding_gen
gc.collect()

89

In [17]:
embeddings.shape

(995, 384)

In [20]:
from sklearn.mixture import GaussianMixture
import numpy as np

class TopicModel(object):

    def __init__(self,n_topics=20):
        self.model = GaussianMixture(n_components=n_topics,random_state=777)


    def _fit_model(self,embeddings):
        self.model.fit(embeddings)

    
    def _select_topics(self,embeddings,save=True):
        probs = self.model.predict_proba(embeddings)
        topics = [np.where(p > 0) for p in probs]
        if save:
            self.topics = topics
            self.probs = probs.tolist()
    

    def get_topics(self,embeddings,save=True):
        if save == True:
            self._fit_model(embeddings)
        self._select_topics(embeddings,save)
        return self.topics

In [21]:
topic_model  = TopicModel()
topics = topic_model.get_topics(embeddings)

C:\Users\juan9\AppData\Roaming\Python\Python38\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


In [22]:
topics

[(array([18], dtype=int64),),
 (array([18], dtype=int64),),
 (array([17], dtype=int64),),
 (array([18], dtype=int64),),
 (array([8], dtype=int64),),
 (array([18], dtype=int64),),
 (array([18], dtype=int64),),
 (array([17], dtype=int64),),
 (array([17], dtype=int64),),
 (array([17], dtype=int64),),
 (array([17], dtype=int64),),
 (array([18], dtype=int64),),
 (array([18], dtype=int64),),
 (array([18], dtype=int64),),
 (array([0], dtype=int64),),
 (array([3], dtype=int64),),
 (array([18], dtype=int64),),
 (array([8], dtype=int64),),
 (array([18], dtype=int64),),
 (array([12], dtype=int64),),
 (array([0], dtype=int64),),
 (array([17], dtype=int64),),
 (array([18], dtype=int64),),
 (array([17], dtype=int64),),
 (array([18], dtype=int64),),
 (array([18], dtype=int64),),
 (array([18], dtype=int64),),
 (array([17], dtype=int64),),
 (array([18], dtype=int64),),
 (array([8], dtype=int64),),
 (array([8], dtype=int64),),
 (array([18], dtype=int64),),
 (array([0], dtype=int64),),
 (array([18], dtyp

In [23]:
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize


class TopicUtilities(object):

    def __init__(self,coherence_mode="c_npmi",n_topics=20):
        self.coherence_mode = coherence_mode
        self.n_topics = n_topics


    def create_utility_objects(self,data):
        self.tokenized_word_sentences = [word_tokenize(s) for s in data.values]
        self.id2word = corpora.Dictionary(self.tokenized_word_sentences)

    

    def get_top_topic_tokens(self,topics):
        topic_top_n = list()
        for topic in range(self.n_topics):
            topic_indexes = [i for i,t in enumerate(topics) for p in t if topic in p]
            topic_sentences = [t for i,t in enumerate(self.tokenized_word_sentences) if i in topic_indexes]
            all_topic_words = [word for sentence in topic_sentences for word in sentence]
            frequency = FreqDist(all_topic_words)
            top_n = frequency.most_common(10)
            top_n_words = [t[0] for t in top_n]
            topic_top_n.append(top_n_words)
        
        self.topic_top_n = topic_top_n
        return topic_top_n
    

    def get_coherence(self,top_tokens):
        cm = CoherenceModel(topics=top_tokens,texts = self.tokenized_word_sentences, dictionary=self.id2word, coherence=self.coherence_mode,processes=1,topn=10)
        coherence = cm.get_coherence()
        return coherence

    

    



In [24]:
gc.collect()

28

In [25]:
utils = TopicUtilities()
utils.create_utility_objects(prepared_text_en_science)
top_tokens = utils.get_top_topic_tokens(topics)
coherence = utils.get_coherence(top_tokens)

In [26]:
coherence

0.0593007704039384

In [27]:
embedding_gen = TextEmbeddingGenerator(prepared_text_es_science,"paraphrase-multilingual-MiniLM-L12-v2")
spa_embeddings = embedding_gen.calculate_embeddings()
embedding_gen.unload_transformer()
del embedding_gen

In [28]:
gc.collect(
)

29

In [30]:
spanish_topics = topic_model.get_topics(spa_embeddings,save=False)

In [31]:
utils = TopicUtilities()
utils.create_utility_objects(prepared_text_es_science)
top_tokens = utils.get_top_topic_tokens(spanish_topics)
coherence = utils.get_coherence(top_tokens)

In [32]:
coherence

0.05995132227614612